# read csv

In [18]:
import pandas as pd

# input files
filename = "./sample data/feature.csv"
filename1 = "./sample data/wq0118_records.csv"
filename3 = "./sample data/wp0071_records.csv"
filename4 = "./sample data/R047_levels.csv"

filename2 = "./sample data/R047_R258_titration_records.csv"

# output directory
file_directory = './sample data/output_3_files.csv'
output_all_data = './sample data/output_all_data.csv'
outpuut_directory_all = './sample data/output_all_4_data.csv'
outpuut_directory_open = './sample data/output_open_4_data.csv'

In [2]:
# read csv
df_feature = pd.read_csv(filename, parse_dates=['timestamp'])
df1_0118 = pd.read_csv(filename1, parse_dates=['measurement_result_date'])
df3_0071 = pd.read_csv(filename3,parse_dates=[['Measurement Result Date','Minute 5']])
df4_047 = pd.read_csv(filename4, parse_dates=['measurement_result_date'], dayfirst=True)

# print shape
print ("feature (df) shape: ", df_feature.shape)
print ("wq0118 (df1) shape: ", df1_0118.shape)
print ("wp0071 (df3) shape: ", df3_0071.shape)
print ("R047 (df4) shape: ", df4_047.shape)

('feature (df) shape: ', (17282, 392))
('wq0118 (df1) shape: ', (192747, 5))
('wp0071 (df3) shape: ', (930222, 27))
('R046 (df4) shape: ', (8640, 5))


In [3]:
# is unique
print ("feature (df): ", df_feature['timestamp'].is_unique)
print ("wq0118 (df1): ", df1_0118['measurement_result_date'].is_unique)
print ("wp0071 (df3): ", df3_0071['Measurement Result Date_Minute 5'].is_unique)
print ("R047 (df4): ", df4_047['measurement_result_date'].is_unique)

('feature (df): ', False)
('wq0118 (df1): ', True)
('wp0071 (df3): ', False)
('R046 (df4): ', True)


### filter duplicated data 

#### wp0071 - pump

In [4]:
# tem_0071 = df3_0071[['Measurement Result Date_Minute 5','Average Result Value', 'Asset Code']].sort_values(['Measurement Result Date_Minute 5'])

In [5]:
# data = tem_0071[930000:930050]
# data.groupby(['Measurement Result Date_Minute 5']).agg({'Average Result Value': 'sum'})

In [6]:
# sum average result value
feature_pmp = df3_0071.groupby(['Measurement Result Date_Minute 5']).agg({'Average Result Value': 'sum'})
# feature_pmp['Average Result Value']

In [7]:
# change index to specific column
feature_pmp['Measurement_Result_Date_Minute_15'] = feature_pmp.index
feature_pmp.shape

(310074, 2)

#### New feature: Average Result Value - indicator for flow speed when pump open 

In [8]:
feature_pmp['Average Result Value'].describe()

count    310074.000000
mean          0.817137
std           1.260779
min           0.000000
25%           0.000000
50%           0.000000
75%           1.000000
max           5.000000
Name: Average Result Value, dtype: float64

In [9]:
feature_pmp['Measurement_Result_Date_Minute_15'].is_unique

True

#### features

In [10]:
features_32138 = df_feature.loc[df_feature['node_id']==31238]
features_32138['timestamp'].describe()

count                    8641
unique                   8641
top       2017-01-29 08:15:00
freq                        1
first     2016-12-01 00:00:00
last      2017-03-01 00:00:00
Name: timestamp, dtype: object

In [11]:
features_32138['timestamp'].is_unique

True

In [12]:
features_41908 = df_feature.loc[df_feature['node_id']==41908]
features_41908['timestamp'].describe()

count                    8641
unique                   8641
top       2017-01-29 08:15:00
freq                        1
first     2016-12-01 00:00:00
last      2017-03-01 00:00:00
Name: timestamp, dtype: object

### New feature: water level
- 25/09/2017

In [40]:
df4_047 = pd.read_csv('./sample data/R047_levels.csv', parse_dates=['measurement_result_date'],dayfirst=True)
df4_047['measurement_result_date'].describe()
# drop useless column
df4_047 = df4_047.drop(['statistic_type', 'asset_code', 'facility_code'], axis=1)
df4_047.columns = ['measurement_result_date', 'water_level']
df4_047 = df4_047.sort_values(['measurement_result_date'])

In [42]:
df4_047['measurement_result_date'].describe()

count                    8640
unique                   8640
top       2017-01-29 08:15:00
freq                        1
first     2016-12-01 00:00:00
last      2017-02-28 23:45:00
Name: measurement_result_date, dtype: object

# merge (inner join)

In [19]:
# merge label and pump state
df_merge_1 = pd.merge(df1_0118, feature_pmp, left_on='measurement_result_date', right_on='Measurement_Result_Date_Minute_15',how='inner')
df_merge_1.shape

(192618, 7)

In [20]:
df_merge_1['measurement_result_date'].is_unique

True

In [21]:
# merge df with features where node_id = 32138 (same with 41908)
df_merge_3 = pd.merge(df_merge_1, features_32138, left_on='measurement_result_date', right_on='timestamp',how='inner')
df_merge_3.shape

(8641, 399)

In [22]:
df_merge_3['Average Result Value'].describe()

count    8641.000000
mean        0.322301
std         0.471330
min         0.000000
25%         0.000000
50%         0.000000
75%         1.000000
max         2.000000
Name: Average Result Value, dtype: float64

In [23]:
df_pump_open = df_merge_3.loc[df_merge_3['Average Result Value']>0]
df_pump_open.shape

(2769, 399)

In [24]:
df_pump_open.sort_values(['measurement_result_date']).head(5)

,measurement_result_date,average_result_value,statistic_type,asset_code,facility_code,Average Result Value,Measurement_Result_Date_Minute_15,zone_flow_hist_0min,zone_flow_hist_15min,zone_flow_hist_30min,...,res_cl_hist_2835min,res_cl_hist_2850min,res_cl_hist_2865min,res_cl_hist_2880min,res_temperature,travel_time_hrs_nom,flow_nom,timestamp,node_id,zone_id
0,2016-12-01 00:00:00,1.4572,Snap (Current),WQM01,WQ0118,1,2016-12-01 00:00:00,0.537967,0.677703,0.838600,...,1.711658,1.709118,1.707396,1.708366,17.4,2.789096,1.931739,2016-12-01 00:00:00,31238.0,engadine_rem_flow_zone
42,2016-12-01 00:15:00,1.4560,Snap (Current),WQM01,WQ0118,1,2016-12-01 00:15:00,0.374730,0.537967,0.677703,...,1.714862,1.711658,1.709118,1.707396,17.4,2.789096,1.931739,2016-12-01 00:15:00,31238.0,engadine_rem_flow_zone
4088,2016-12-01 00:30:00,1.4540,Snap (Current),WQM01,WQ0118,1,2016-12-01 00:30:00,0.228371,0.374730,0.537967,...,1.718593,1.714862,1.711658,1.709118,17.4,2.789096,1.931739,2016-12-01 00:30:00,31238.0,engadine_rem_flow_zone
125,2016-12-01 00:45:00,1.4432,Snap (Current),WQM01,WQ0118,1,2016-12-01 00:45:00,0.169935,0.228371,0.374730,...,1.720893,1.718593,1.714862,1.711658,17.4,2.789096,1.931739,2016-12-01 00:45:00,31238.0,engadine_rem_flow_zone
167,2016-12-01 01:00:00,1.4306,Snap (Current),WQM01,WQ0118,1,2016-12-01 01:00:00,0.172979,0.169935,0.228371,...,1.721563,1.720893,1.718593,1.714862,17.4,2.789096,1.931739,2016-12-01 01:00:00,31238.0,engadine_rem_flow_zone


## output data by merging 4 files
- 25/09/2018

In [47]:
# merge 3 files with water level
df_merge_4 = pd.merge(df_merge_3, df4_047, left_on='measurement_result_date', right_on='measurement_result_date',how='inner')
print "Output of merging all 4 files:",df_merge_4.shape

out put of merging all 4 files: (8640, 400)


In [45]:
df_merge_4['measurement_result_date'].sort_values().describe()

count                    8640
unique                   8640
top       2017-01-29 08:15:00
freq                        1
first     2016-12-01 00:00:00
last      2017-02-28 23:45:00
Name: measurement_result_date, dtype: object

## output open data by filter all data of 4 files
- 25/09/2018

In [48]:
df_pump_open_4 = df_merge_3.loc[df_merge_3['Average Result Value']>0]
print "Output of opening data with water level:", df_pump_open_4.shape

Output of opening data with water level: (2769, 399)


# save file - 1

#### sort values by daytime and output


In [21]:
# 8641 - dataset in 3 months
df_merge_3.sort_values(['measurement_result_date']).to_csv(output_all_data,index=None)

# 2769 - dataset in 3 months only when pump is open
df_pump_open.sort_values(['measurement_result_date']).to_csv(file_directory, index=None)

## save file - 2
- 25/09/2018

In [49]:
# 8640 - dataset in 3 months from 4 files
df_merge_4.sort_values(['measurement_result_date']).to_csv(outpuut_directory_all, index=None)

# 2769 - dataset in 3 months only when pump is open
df_pump_open_4.sort_values(['measurement_result_date']).to_csv(outpuut_directory_open, index=None)

# plot 

In [20]:
x_axis= df_pump_open['measurement_result_date']
y_label = df_pump_open['average_result_value']
y_zone_cl = df_pump_open['zone_flow_hist_0min']
y_res_cl = df_pump_open['res_cl_hist_0min']

In [21]:
%matplotlib
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(20, 10))

ax1 = fig.add_subplot(111)
ax1.plot(x_axis, y_label, color='red', label='average_result_value')
ax1.plot(x_axis, y_res_cl, color='blue', label='res_cl_hist_0min')
ax1.set_ylabel('chlorine')
ax1.set_title("data visualization")

ax2 = ax1.twinx()  # this is the important function
ax2.plot(x_axis, y_zone_cl, 'r', color='green', label='zone_flow_hist_0min')
ax2.set_ylabel('folw')
ax2.set_xlabel('time')

plt.grid(True)
 
plt.legend(bbox_to_anchor=(1.0, 1), loc=1, borderaxespad=0.)

plt.show()

Using matplotlib backend: Qt5Agg


In [22]:
from pandas import DataFrame
df_drop_plot = DataFrame()
df_drop_plot['measurement_result_date'] = df_pump_open['measurement_result_date']
df_drop_plot['average_result_value'] = df_pump_open['average_result_value']
df_drop_plot.plot(x='measurement_result_date', y='average_result_value')
plt.show()

In [23]:
from pandas import DataFrame

df_plot_1 = DataFrame()
df_plot_1= df_pump_open[['average_result_value','res_cl_hist_0min','measurement_result_date','zone_flow_hist_0min']]
df_plot_1.index = df_plot_1['measurement_result_date']
df_plot = df_plot_1.drop(['measurement_result_date'],axis=1)

df_plot.sort_values(['average_result_value']).head(5)

,average_result_value,res_cl_hist_0min,zone_flow_hist_0min
measurement_result_date,,,
2017-01-08 22:00:00,0.7450,1.377217,2.183595
2017-01-08 22:15:00,0.7476,1.374898,1.959374
2017-01-08 21:30:00,0.7496,1.381701,2.826537
2017-01-08 21:45:00,0.7558,1.379573,2.450470
2017-01-08 21:15:00,0.7584,1.383780,3.299814


In [24]:
df_plot['average_result_value'].describe()

count    2769.000000
mean        1.276345
std         0.139701
min         0.745000
25%         1.197000
50%         1.290200
75%         1.369600
max         1.617200
Name: average_result_value, dtype: float64

In [25]:
%matplotlib
import matplotlib.pyplot as plt

df_plot.plot()

Using matplotlib backend: Qt5Agg


In [26]:
df_plot.plot(secondary_y=['zone_flow_hist_0min'])

In [30]:
a = df_plot.sort_values(['average_result_value'])

In [31]:
a[1000:1030]

,average_result_value,res_cl_hist_0min,zone_flow_hist_0min
measurement_result_date,,,
2017-01-26 03:00:00,1.2440,1.441586,0.298297
2017-01-17 02:15:00,1.2440,1.478370,0.255367
2017-02-20 04:15:00,1.2440,1.313414,0.048853
2016-12-06 23:45:00,1.2440,1.496672,0.325878
2017-01-22 03:30:00,1.2440,1.488384,0.494244
2017-01-05 03:15:00,1.2440,1.421695,0.073985
2017-01-16 02:45:00,1.2452,1.448487,0.132165
2017-01-16 23:15:00,1.2452,1.447434,1.266805
2017-01-23 00:00:00,1.2452,1.476316,0.847696
